In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [1]:
import pickle

with open('URM_train_validation.pkl', 'rb') as f:
    URM_train_validation = pickle.load(f)
with open('URM_test.pkl', 'rb') as f:
    URM_test = pickle.load(f)

In [11]:
from Hybrid import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
## SLIM
slim = SLIMElasticNetRecommender(URM_train_validation)
slim.load_model("slim_models", "slim_t_val")
## RP3BETA
rp3beta = RP3betaRecommender(URM_train_validation)
rp3beta.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
## P3ALPHA
p3 = P3alphaRecommender(URM_train_validation)
p3.fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)

## EASE
ease = EASE_R_Recommender(URM_train_validation)
ease.load_model("slim_models", "ease_t_val")
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

SLIMElasticNetRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file 'slim_modelsslim_t_val'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 5797.10 column/sec. Elapsed time 3.83 sec
P3alphaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 5567.73 column/sec. Elapsed time 3.99 sec
EASE_R_Recommender: URM Detected 205 ( 1.6%) users with no interactions.
EASE_R_Recommender: URM Detected 109 ( 0.5%) items with no interactions.
EASE_R_Recommender: Loading model from file 'slim_modelsease_t_val'
EASE_R_Recommender: Loa

In [6]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2203 (17.4%) Users that have less than 1 test interactions


In [3]:
import numpy as np 
import scipy.sparse as sps
profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)

#profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)
profile_length, profile_length.shape

(array([36, 28,  3, ...,  5, 20, 56], dtype=int32), (12638,))

In [12]:
# PROFILE LENGTH BASED GROUPING
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

MAP_recommender_per_group = {}
cutoff = 10
threshold = 45

# Select users whose profile length is less than the cutoff for the group and greater than or equal to the cutoff for the previous group

users_in_group_flag = profile_length > threshold
users_in_group = np.arange(len(profile_length))[users_in_group_flag]

users_in_group_p_len = profile_length[users_in_group]

# Select users not in the current group
users_not_in_group_flag = np.isin(np.arange(len(profile_length)), users_in_group, invert=True)
users_not_in_group = np.arange(len(profile_length))[users_not_in_group_flag]

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)

EvaluatorHoldout: Ignoring 2203 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10070 Users


In [13]:
import optuna as op
hybrid = LinearHybridRecommender(URM_train_validation, [slim, rp3beta, p3, ease])

def objective(trial):
    alpha = trial.suggest_float('alpha', 0, 1)
    beta = trial.suggest_float('beta', 0, 1)
    gamma = trial.suggest_float('gamma', 0, 1)
    delta = trial.suggest_float('delta', 0, 1)
    # Create hybrid recommender with the suggested weights
    hybrid.fit(alphas=[alpha, beta, gamma, delta])
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid)
    MAP_result = result["MAP"].item()
       
    return MAP_result

SLIMElasticNetRP3betaP3alphaEASE_R_LinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaP3alphaEASE_R_LinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [20]:
best_params = {
    "alpha": 1,
}
study_name = "hybrid-sl-rp3-p3-ease-veryhi-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2024-01-08 16:08:04,089] Using an existing study with name 'hybrid-sl-rp3-p3-ease-veryhi-study' instead of creating a new one.


EvaluatorHoldout: Processed 2568 (100.0%) in 3.31 sec. Users per second: 776


[I 2024-01-08 16:08:07,539] Trial 600 finished with value: 0.1050013443851061 and parameters: {'alpha': 0.9874883060173058, 'beta': 0.9851180135425676, 'gamma': 0.6275093126626894, 'delta': 0.15610106044018784}. Best is trial 437 with value: 0.10609228971962607.


EvaluatorHoldout: Processed 2568 (100.0%) in 2.93 sec. Users per second: 878


[I 2024-01-08 16:08:10,509] Trial 601 finished with value: 0.10553733681946285 and parameters: {'alpha': 0.6435795890832501, 'beta': 0.2642478746571836, 'gamma': 0.35686672130995856, 'delta': 0.1279139930224713}. Best is trial 437 with value: 0.10609228971962607.


...

[I 2024-01-08 16:12:51,434] Trial 697 finished with value: 0.10416674393017858 and parameters: {'alpha': 0.5051260505225957, 'beta': 0.7789519195649763, 'gamma': 0.7108776411620928, 'delta': 0.09089350432827509}. Best is trial 437 with value: 0.10609228971962607.


EvaluatorHoldout: Processed 2568 (100.0%) in 3.47 sec. Users per second: 741


[I 2024-01-08 16:12:54,953] Trial 698 finished with value: 0.10426771188745493 and parameters: {'alpha': 0.579970329526471, 'beta': 0.26376128981692004, 'gamma': 0.22985198859058112, 'delta': 0.5558073387116506}. Best is trial 437 with value: 0.10609228971962607.


EvaluatorHoldout: Processed 2568 (100.0%) in 2.79 sec. Users per second: 919


[I 2024-01-08 16:12:57,796] Trial 699 finished with value: 0.10419428064579923 and parameters: {'alpha': 0.922491681050054, 'beta': 0.6018233849824389, 'gamma': 0.8260124632267317, 'delta': 0.7526067027436315}. Best is trial 437 with value: 0.10609228971962607.


In [16]:
study.best_params

{'alpha': 0.8857667100747117,
 'beta': 0.6379807942443021,
 'gamma': 0.0690929184825888,
 'delta': 0.11953478623354052}